In [ ]:
%cd ../

In [ ]:
import time

import plasma.parallel_processing.communicators as ppc
import plasma.parallel_processing.queues as ppq
from plasma.functional import partials, AutoPipe, LambdaPipe

In [ ]:
def compute(x):
    time.sleep(0.5)
    return f'compute: {x}'

def compute2(x):
    time.sleep(0.5)
    return f'compute2: {x + 'c'}'

class ComputeClass(AutoPipe):

    def __init__(self, hihi, haha):
        super().__init__()
    
    def run(self, tmp):
        return 'ComputeClass: ' + tmp + 'd'

In [ ]:
def input_handler(i):
    return [i] * 2

In [ ]:
out_queue= ppq.ThreadQueue().register_callback(print)
temp = partials(print, 'test')

class Flow(ppc.AsyncFlow):

    def __init__(self):
        super().__init__()

        distributor = ppc.distributors.IteratorDistributor()
        compute3 = ComputeClass(1, 2)
        self.chain(
            (ppq.ThreadQueue(), input_handler, distributor),
            (input_handler, compute, ppq.PseudoQueue()),
                (compute, print, ppq.PseudoQueue()),
                (compute, out_queue),
                (compute, compute2, ppq.PseudoQueue()),
                    (compute2, print),
                    (compute2, out_queue),
                    (compute2, compute3, ppq.PseudoQueue()),
                        (compute3, print),
                        (compute3, out_queue)
        )

flow = Flow()
flow

In [ ]:
with flow:
   for i in range(10):
      flow.put(i)

In [ ]:
flow.alive()

In [ ]:
out_queue.run()

In [ ]:
out_queue.release()